## Situation and Task
Client demands volatilities metric for bond optionality, while these options are not traded.

- Either for user to **better gauge the pure credit risk in spreads**, or **serves as a relative value metric**.
- **Difficulty**: 
    - There is **no standalone observed swaptions** for sectors with many callable bonds, such as corp HY and munis. 
    - What is worse, issuers calling these bonds **not purely for interest-rate reasons**.

## Action

Build a parsimonious, joint model for interest-rate trees and parametric model of default term-structure, and use BVAL prices to calibrate for both vol and credit spread parameters.

### Main Methodology
- For any given volatility, one can build an interest-rate (short-rate) tree like the usual OAS model, **taking all other parameters as the swaption one except for the vol (e.g. mean-reversion speed in Black-Karacinski)**. 
- Overlay this interest rate tree with the **survival probability by the exponential spline** (see this [notebook](../../fixed-income/instruments/gsac/credit-pricing-and-analytics.ipynb)), we can then discount that cashflow in that branch (scenario), and further PV the bond for the whole tree. 
- PV is thus a function of the credit parameters (exponential spline) and interest rate vol, and we can **solve for an interest rate vol that matches the PV to the bond price**.
    - A somewhat more detailed description of the optimization algorithm is, for any fixed interest rate vol, the PV is a linear function of the exponential spline parameters. Thus given the vol, **solving for the exponential spline coefficients matching PV to bond prices is again solving a linear regression**.
    - After obtaining the minimal pricing errors in the regression above, we have a **one-dimensional pricing-error function** of interest rate vol, which is easy to optimize as well.

### Further Difficulty
**Data are uneven cross-sectionally**, in that some issuers may have more bonds and more liquid observations - apparently this methodology does not apply to all issuers.

### Response
We confined ourselves to **a few HY active issuers (such as CHTR)**, and **rating-level volatility calibration for munis** - in fact, we advertised BVAL is going to produce for AAA 5% the bullet curve.

## Resolution
- Clients love the notion and it is on track to become BVAL's new income source.
- But actually, from a hedging perspective, marking implied vol is probably less useful; one should try to **imply the issuer's effective call prices on each call date**.
    - The peculiar constraints that issuers have in calls, that are othogonal to pure interest-rate economics, will **manifest in the implied call prices, rather than in interest rate** - the implied call prices thus have better economic meaning.
    - Once the payoffs of the contigent claim of the call option is determined by the implied call prices, the **hedging should be more precise**; see also relating issue in this [notebook](../cds-bond-basis/basis-callable.ipynb).